In [1]:
import pandas as pd

In [2]:
contents = open("1a22_pdbsum_output.txt",mode="r").readlines()

In [3]:
contents = [x.strip() for x in contents]

In [4]:
contents

['List of atom-atom interactions across protein-protein interface',
 '---------------------------------------------------------------',
 '',
 'PDB code: 1a22   Chains A }{ B',
 '------------------------------',
 '',
 '',
 '',
 'Hydrogen bonds',
 '--------------',
 '',
 '<----- A T O M   1 ----->       <----- A T O M   2 ----->',
 '',
 'Atom Atom Res  Res              Atom Atom Res  Res',
 'no. name name no.  Chain        no. name name no.  Chain  Distance',
 '1.    351  NZ  LYS   41    A   <-->  2170  OE2 GLU  327    B      3.04',
 '2.    384  O   LEU   45    A   <-->  1761  NE1 TRP  276    B      2.65',
 '3.    397  NE2 GLN   46    A   <-->  2114  OE2 GLU  320    B      3.05',
 '4.    474  OE2 GLU   56    A   <-->  1722  NH2 ARG  271    B      2.88',
 '5.    513  O   SER   62    A   <-->  1975  N   ILE  303    B      2.71',
 '6.    531  NE  ARG   64    A   <-->  2434  OD2 ASP  364    B      2.90',
 '7.    534  NH2 ARG   64    A   <-->  2434  OD2 ASP  364    B      3.27',
 '8.    568  

In [5]:
hbonds = False
hbonds_data = []
nonbonded = False
nonbonded_data = []
salt_bridge = False
salt_bridge_data = []
for line in contents:
    if line == "Hydrogen bonds":
        hbonds = True
        nonbonded = False
        salt_bridge = False
    elif line == "Non-bonded contacts":
        hbonds = False
        nonbonded = True
        salt_bridge = False
    elif line == "Salt bridges":
        hbonds = False
        nonbonded = False
        salt_bridge = True
    if hbonds and len(line)!=0:
        text = line.split()
        text.append("Hydrogen bond")
        hbonds_data.append(text)
    if nonbonded and len(line)!=0:
        text = line.split()
        text.append("Hydrophobic")
        nonbonded_data.append(text)
    if salt_bridge and len(line)!=0:
        text = line.split()
        text.append("Salt bridge")
        salt_bridge_data.append(text)

In [6]:
interactions = pd.DataFrame(hbonds_data[5::] + nonbonded_data[5::] + salt_bridge_data[5::])
interactions.columns = ["Interaction no.","Atom 1 no.","Atom 1 name","Res 1 name","Res 1 no.","Chain 1","---","Atom 2 no.","Atom 2 name","Res 2 name","Res 2 no.","Chain 2","Distance","Interaction type"]
interactions = interactions.drop(["Interaction no.","---"],axis=1)
interactions = interactions[["Interaction type","Atom 1 name","Res 1 name","Res 1 no.","Atom 2 name","Res 2 name","Res 2 no.","Distance"]]
interactions.index = [x+1 for x in interactions.index]

In [7]:
interactions

,Interaction type,Atom 1 name,Res 1 name,Res 1 no.,Atom 2 name,Res 2 name,Res 2 no.,Distance
1,Hydrogen bond,NZ,LYS,41,OE2,GLU,327,3.04
2,Hydrogen bond,O,LEU,45,NE1,TRP,276,2.65
3,Hydrogen bond,NE2,GLN,46,OE2,GLU,320,3.05
4,Hydrogen bond,OE2,GLU,56,NH2,ARG,271,2.88
5,Hydrogen bond,O,SER,62,N,ILE,303,2.71
...,...,...,...,...,...,...,...,...
168,Salt bridge,OE2,GLU,56,NH2,ARG,271,2.88
169,Salt bridge,NH1,ARG,64,OE1,GLU,244,3.76
170,Salt bridge,NE,ARG,64,OD2,ASP,364,2.90
171,Salt bridge,NH2,ARG,167,OE2,GLU,327,3.04


In [8]:
interaction_number = {(interactions.loc[i]["Res 1 name"]+interactions.loc[i]["Res 1 no."],interactions.loc[i]["Res 2 name"]+interactions.loc[i]["Res 2 no."]):0 for i in range(1,len(interactions)+1)}
for i in range(1,len(interactions)+1):
    interaction_number[(interactions.loc[i]["Res 1 name"]+interactions.loc[i]["Res 1 no."],interactions.loc[i]["Res 2 name"]+interactions.loc[i]["Res 2 no."])] += 1

In [9]:
interaction_number = {pair:contacts for pair,contacts in sorted(interaction_number.items(), key=lambda item: item[1],reverse=True)}

In [10]:
for pair,contacts in interaction_number.items():
    print(pair,contacts)

('HIS18', 'ARG417') 11
('ARG64', 'TRP369') 11
('ARG64', 'ASP364') 9
('HIS18', 'ASN418') 8
('LEU45', 'TRP276') 7
('GLN46', 'TRP276') 7
('GLU56', 'ARG271') 6
('SER62', 'ILE303') 6
('HIS21', 'ASN418') 6
('LYS41', 'GLU327') 5
('ARG167', 'GLU327') 5
('THR175', 'ARG243') 5
('TYR42', 'CYS322') 5
('ARG64', 'GLU244') 5
('ASP171', 'ARG243') 5
('GLN68', 'ASP364') 4
('SER62', 'SER302') 4
('THR67', 'TRP369') 4
('ARG178', 'GLY368') 4
('GLN46', 'GLU320') 3
('LYS168', 'TRP304') 3
('PHE25', 'ASN418') 3
('LEU52', 'ARG271') 3
('ARG64', 'LYS367') 3
('GLN68', 'LYS367') 3
('ASP171', 'TRP304') 3
('GLN22', 'ASN418') 2
('GLN46', 'CYS308') 2
('PRO48', 'TRP276') 2
('PRO61', 'TRP304') 2
('ASN63', 'TRP369') 2
('LYS172', 'TRP304') 2
('GLU174', 'ASN418') 2
('CYS182', 'GLY368') 2
('HIS21', 'SER419') 1
('LYS41', 'CYS322') 1
('LEU45', 'PRO306') 1
('GLN46', 'CYS322') 1
('PRO48', 'GLN274') 1
('PRO48', 'GLU275') 1
('LEU52', 'GLN274') 1
('GLN68', 'TRP369') 1
('ASP171', 'ASP326') 1
('ASP171', 'SER419') 1
('THR175', 'TRP369'